---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [5]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
import re

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [6]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [7]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    town_f = open("university_towns.txt","r",encoding='utf8')
    state=""
    city=""
    town_list=[]
    stateMatch = re.compile(".*\[edit].*")

    for line in town_f:
        # Can't resist the most basic of data cleansing!
        line = re.sub("\n","",line)

        if stateMatch.match( line ):
            state = line
        else:
            city = line

        if state != "" and city != "":
            town_list += [[state, city]]
            city = ""

    towns = pd.DataFrame( town_list, columns=["State", "RegionName"])
    towns['State'] = towns['State'].map(lambda x : re.sub("(\[.*?])", "", x))
    towns['RegionName'] = towns['RegionName'].map(lambda x : re.sub("(\[.*?])", "", re.sub(" (\(.*?$)", "", x)))
    
    return towns

In [8]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    gdp = pd.read_excel('gdplev.xls', sheetname='Sheet1', skiprows=7,parse_cols="E:G",header=0,index_col=0,
                        names=['gdpd','gdpcd'])
    gdp = gdp[(gdp.index > '2000aa')]
    gdp['gdp_diff']=gdp['gdpcd'].diff().shift(-1)
    #print(gdp)
    
    for a in range(len(gdp)):
        #print(gdp.iloc[a])
        if (gdp.iloc[a]['gdp_diff'] < 0) and (gdp.iloc[a-1]['gdp_diff'] < 0):
            return gdp.iloc[a].name
    
    return ""

In [9]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    gdp = pd.read_excel('gdplev.xls', sheetname='Sheet1', skiprows=7,parse_cols="E:G",header=0,index_col=0,
                        names=['gdpd','gdpcd'])
    gdp = gdp[(gdp.index > '2000aa')]
    gdp['gdp_diff']=gdp['gdpcd'].diff()
    #print(gdp)
    
    recessionStarted = False
    
    for a in range(len(gdp)):
        #print(gdp.iloc[a])
        if (gdp.iloc[a]['gdp_diff'] < 0) and (gdp.iloc[a-1]['gdp_diff'] < 0):
            recessionStarted = True
            #print( "Recession started: ", gdp.iloc[a-1].name )
        
        if (gdp.iloc[a]['gdp_diff'] > 0) and (gdp.iloc[a-1]['gdp_diff'] > 0) and recessionStarted:
            return gdp.iloc[a].name        
    
    return ""

In [ ]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    gdp = pd.read_excel('gdplev.xls', sheetname='Sheet1', skiprows=7,parse_cols="E:G",header=0,index_col=0,
                        names=['gdpd','gdpcd'])
    gdp = gdp[(gdp.index > '2000aa')]
    gdp['gdp_diff']=gdp['gdpcd'].diff()
    #print(gdp)
    
    recessionStarted = False
    minGDP = 0
    minGDPi = 0
    
    for a in range(len(gdp)):
        #print(gdp.iloc[a])
        if (gdp.iloc[a]['gdp_diff'] < 0) and (gdp.iloc[a-1]['gdp_diff'] < 0):
            recessionStarted = True
            minGDP=gdp.iloc[a]['gdpcd']
            minGDPi=a
            #print( "Recession started: ", gdp.iloc[a-1].name )
        
        if recessionStarted and (minGDP > gdp.iloc[a]['gdpcd']):
            minGDP=gdp.iloc[a]['gdpcd']
            minGDPi=a
        
        if (gdp.iloc[a]['gdp_diff'] > 0) and (gdp.iloc[a-1]['gdp_diff'] > 0) and recessionStarted:
            return gdp.iloc[minGDPi].name      
    
    return ""

In [13]:
def get_month_headings( yr, qtr, valid_months):
    month_headings = []
    
    for count in range(1,4):
        if qtr == 4:
            mth=str(9+count)
        else:
            mth='0'+str((qtr-1)*3+count)
            
        curr_mth = str(yr)+'-'+mth
        if (curr_mth in valid_months):
            month_headings += [curr_mth]
    
    return month_headings

def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    workDF = pd.read_csv('City_Zhvi_AllHomes.csv')
    
    # The instructions don't state it but the state code must be mapped to a
    # state name using the states dictionary
    workDF['State'] = workDF['State'].map(lambda x: states.get(x))
    workDF = workDF.set_index(['State','RegionName'])
    
    # Throw away the first 4 useless columns
    years=workDF.columns[4:]
    
    # We're going to be throwing away columns so let's make a list of those to keep
    cols_to_keep=[]

    # We need to find which quarters we need to process, starting in year 2000
    minYear = "2000"
    maxYear = re.sub("(-..)", "", years[len(years)-1])

    for year in range(int(minYear),int(maxYear)+1):
        for quarter in range(1,4+1):
            mth = str((quarter-1)*3+1) if ((quarter-1)*3+1 >= 10) else '0'+str((quarter-1)*3+1)
            
            # Does this quarter exist in the dataset?
            if (str(year)+'-'+mth in years):
                
                # Get the mean of the months in the quarter and write in YYYYqQ formatted column
                workDF[str(year)+'q'+str(quarter)] = workDF[get_month_headings(year,quarter,years)].mean(axis=1)
                
                # We'll want to keep this new column we just created
                cols_to_keep += [str(year)+'q'+str(quarter)]
    
    housingDF = workDF[cols_to_keep]
    return housingDF

In [1]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''

    # Load the housing data
    housingDF = convert_housing_data_to_quarters()
    # Calculate the Price Ratio
    housingDF['Price Ratio'] = housingDF[get_recession_start()].div(housingDF[get_recession_bottom()])

    # Reset the index so we can get rid of duplicate RegionName data
    housingDF = housingDF.reset_index()
    housingDF = housingDF[['State','RegionName','Price Ratio']] 
    housingDF = housingDF.groupby(['State','RegionName']).agg({'Price Ratio': np.average})
    housingDF = housingDF.reset_index()    
    housingDF = housingDF[['State','RegionName','Price Ratio']].dropna()

    # Get the list of university towns with State and RegionName columns
    university_towns = get_list_of_university_towns()
    university_towns = university_towns.reset_index()
    
    # Merge housing and university town data
    mergedData = pd.merge(housingDF, university_towns, how="outer", left_on=['State','RegionName'], 
                          right_on=['State','RegionName'], indicator=True)
    
    # Data found in both datasets is a university town
    uniTowns = mergedData[mergedData["_merge"] == "both"].dropna()
    #print(uniTowns)
    
    # Data only found in the housing dataset is a non-university town
    nonUniTowns = mergedData[mergedData["_merge"] == "left_only"][["State","RegionName","Price Ratio","_merge"]].dropna()
    #print(nonUniTowns)
    
    (stats, pvalue) = ttest_ind(uniTowns['Price Ratio'], nonUniTowns['Price Ratio'])
    
    different = True if pvalue<0.01 else False
    better = "university town"
    
    return (different, pvalue, better)